# Machine Learnig 🤖

Notebook voltado ao estudo do machine learning, área da computação destinado ao aprendizado da máquina, por meio da qual a capacita a entender problemas e cenários, podendo classificá-los, prevê-los, agrupá-los e, com a sua componente generativa, gerar informação.

No presente momento, o estudo de ML irá se debruçar sobre um problema de classificação, no qual irei identificar, a partir de um estudo de caso, clientes que podem ou não adentrar a um cenário de churn (cancelamento de um serviço).

## O que é classificação ? ✒️

Em essência, a classificação é um método com o qual, por meio do ML, permite classificar e prever a qual categoria ou classe um determinado dado pertence. Por exemplo, a partir de determinadas características, quais se relacionam ao grupo dos que apresentam churn e dos que não apresentam ?

É construido por meio da relação das variáveis explicativas e da variável resposta, passando pelo escopo do treinamento, teste (e às vezes validação), por meio da qual, posteriormente, o modelo pode ser utilizado para prever, com base nas variáveis explicativas, a que determinado grupo certos dados representam ou fazem parte.

As variáveis explicativas, também podem ser compreendidos como dados de entrada (X), enquanto que a variável resposta pode ser compreendida como a variável de saida (y).

### Algoritmos de classificação :  👺  

Abaixo alguns algoritmos de classificação que pode-se citar.

- K-Nearest-Neighbors (KNN) ;
- Support Vector Machine (SVM) ;
- Decision Tree Classifier ;
- Random Forest Classifier .  

## Mas e a regressão ❓

Muitas vezes a regressão vem acompanhada em questões de classificação, como classificar o sentimento de uma música entre eufórica e melancólica, através de variáveis explicativas que a compõe.

Porém, ainda que seja efetivo a sua utilização nesse cenários, como por exemplo utilizando a regressão logística, para termos de melhor compreensão / segmentação, podemos compreender que, ao contrário da classificação, a regressão busca lidar em sua essência com a previsão de um valor numérico em específico, como prever preços futuros, estoques, receita futura e etc.

Desse modo, quando se lida com a previsibilidade de termos numéricos, estamos lidando com uma regressão.

### Exemplos de modelos de regressão :     

- Random Forest Regressor ;
- Linear Regression ;
- Logistic Regression .

In [3]:
# Carregando as bibliotecas utilizadas

import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [4]:
# Carregando os dataframes

churn_20 = pd.read_csv('/content/churn-bigml-20.csv')
churn_80 = pd.read_csv('/content/churn-bigml-80.csv')
customer_df = pd.read_csv('/content/Customer-Churn.csv')

In [5]:
# Realizando uma exploração dos dataframes,
# acerca do seu formato, dados duplicados,
# nulos ou faltantes e etc.

print(f'Formato do dataframe churn_20 : {churn_20.shape}')
print(f'Formato do dataframe churn_80 : {churn_80.shape}')

Formato do dataframe churn_20 : (667, 20)
Formato do dataframe churn_80 : (2666, 20)


In [6]:
def verificaDataFrame(df):

  qt_n_disponivel = df.isna().sum().any()
  qt_nulos = df.isnull().sum().any()
  qt_duplicados = df.duplicated().sum().any()

  if qt_n_disponivel:

    print(f'Há dados não disponíveis no DataFrame.')

  if qt_nulos:

    print(f'Há dados nulos no DataFrame.')

  if qt_duplicados:

    print(f'Há dados duplicados no DataFrame.')

  if not any([qt_n_disponivel, qt_nulos, qt_duplicados]):

    print("""Não há dados duplicados, nulos ou não disponíveis em seu DataFrame.
Sinta-se à vontade para utilizá-los em seu projeto e/ou estudo.""")

In [7]:
verificaDataFrame(churn_20)
print('')
verificaDataFrame(churn_80)
print('')
verificaDataFrame(customer_df)

Não há dados duplicados, nulos ou não disponíveis em seu DataFrame.
Sinta-se à vontade para utilizá-los em seu projeto e/ou estudo.

Não há dados duplicados, nulos ou não disponíveis em seu DataFrame.
Sinta-se à vontade para utilizá-los em seu projeto e/ou estudo.

Há dados duplicados no DataFrame.


In [8]:
churn_20.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [9]:
churn_80.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [10]:
customer_df.columns

Index(['Maior65Anos', 'Conjuge', 'Dependentes', 'MesesDeContrato',
       'TelefoneFixo', 'VariasLinhasTelefonicas', 'ServicoDeInternet',
       'SegurancaOnline', 'BackupOnline', 'SeguroNoDispositivo',
       'SuporteTecnico', 'TVaCabo', 'StreamingDeFilmes', 'TipoDeContrato',
       'PagamentoOnline', 'FormaDePagamento', 'ContaMensal', 'Churn'],
      dtype='object')

Analisando as colunas de cada dataframe, pode-se depreender que os dois primeiros se referem ao mesmo escopo, de modo que, porém, estão segmentados numa proporção 20: 80.

O outro dataframe, referente aos consumidores, apresenta um outro escopo, considerando as características do consumidor, por meio das quais pode haver relação com esse adentrar a um cenário de *churn* (cancelamento do serviço) ou não.

In [11]:
customer_df.shape

(7043, 18)

In [12]:
customer_df.duplicated().value_counts()

False    6985
True       58
Name: count, dtype: int64

In [13]:
print(f'Proporção de dads duplicados no dataframe : {round((58 / 7043), 3)*100} %')

Proporção de dads duplicados no dataframe : 0.8 %


In [14]:
# Como a quantidade de dados duplicados é ínfima perante ao
# todo, irei excluir os dados que são duplicados.

customer_df = customer_df.drop_duplicates()

In [15]:
# Verificando o dataframe novamente:
verificaDataFrame(customer_df)

Não há dados duplicados, nulos ou não disponíveis em seu DataFrame.
Sinta-se à vontade para utilizá-los em seu projeto e/ou estudo.


In [16]:
customer_df.head()

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao
4,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,70.70,Sim


## Analisando as variáveis 🔍

Nesse dataframe referente aos consumidores, temos dois tipos de variáveis que podem ser vistas: a categórica, representada, por exemplo, pela coluna **conjunge**, e a quantitativa, representada pela coluna do valor da **conta mensal**.

>

- V. categórica :    

Variáveis não numéricas, que não possuem um valor numérico intrínseco, ainda que possam ser codificadas como números, e representam categorias ou grupos, podendo apresentar caráter nominal ou ordinal (referente a ordenação, como primeiro, segundo, terceiro e afins)

>

- V. numérica:

Variáveis numéricas são aquelas que possuem um valor numérico intrínseco, representando quantidades ou medidas. Podem ser contínuas (valores fracionados) ou discretas (inteiros)

>

🗯️

Verificando o dataframe, nota-se que há nele muito da presença de variáveis categóricas, referentes tanto a um escopo de sim/ não quanto de tipos/ foramto, como o tipo de contrato e formato de pagamento por exemplo.

Tendo em vista que o ML em sua modelagem, sem utilizar nenhuma camada de PLN, trabalha com termos numéricos, passar a ele variáveis textuais não irá lograr êxito, fazendo-o falhar.

Desse modo, essas precisam ser transformadas, traduzidas de um modo que seja inteligível à máquina. Para tanto, para o presente caso, pode-se tanto mapear algumas variáveis que apresentam relação de sim/ não, quanto de 'dummerizar' aquelas que se referem a tipos/ formatos, de modo a dizer que quando for de um determinado tipo apresentará um valor de 1 e, quando de outro, de 0 e assim por diante.

In [17]:
# Traduzindo as colunas que apresentam relação sim / não
# a uma linguagem que a máquina possa entender melhor:

traducao = {'Sim' : 1,
            'Nao' : 0}

customer_df_mdfy = customer_df[['Conjuge', 'Dependentes',
                                'TelefoneFixo', 'PagamentoOnline',
                                'Churn']].replace(traducao)

customer_df_mdfy.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn
0,1,0,0,1,0
1,0,0,1,0,0
2,0,0,1,1,1
3,0,0,0,0,0
4,0,0,1,1,1


In [18]:
# Dummerizando os dados:
customer_df_dummi = pd.get_dummies(customer_df.drop(['Conjuge', 'Dependentes', 'TelefoneFixo',
                                      'PagamentoOnline', 'Churn'], axis = 1))

# Forçando valores dummirizados a ficar como 1 ou 0,
# pois o get_dummies não estava retornando por padrão
# os dados nesse formato.
customer_df_dummi = customer_df_dummi.astype(int)

customer_df_dummi.head()

,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,VariasLinhasTelefonicas_Sim,ServicoDeInternet_DSL,ServicoDeInternet_FibraOptica,ServicoDeInternet_Nao,SegurancaOnline_Nao,...,StreamingDeFilmes_Nao,StreamingDeFilmes_SemServicoDeInternet,StreamingDeFilmes_Sim,TipoDeContrato_DoisAnos,TipoDeContrato_Mensalmente,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta
0,0,1,29,0,1,0,1,0,0,1,...,1,0,0,0,1,0,0,1,0,0
1,0,34,56,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,0,2,53,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3,0,45,42,0,1,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,0,2,70,1,0,0,0,1,0,1,...,1,0,0,0,1,0,0,1,0,0


In [19]:
# Juntando os dataframes modificados :

customer_df_2 = pd.concat([customer_df_mdfy ,customer_df_dummi], axis = 1)

# Informando a configuração da tabela pandas
# que quero que me retorne todas as colunas presentes.
pd.set_option('display.max_columns', 39)

customer_df_2.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,VariasLinhasTelefonicas_Sim,ServicoDeInternet_DSL,ServicoDeInternet_FibraOptica,ServicoDeInternet_Nao,SegurancaOnline_Nao,SegurancaOnline_SemServicoDeInternet,SegurancaOnline_Sim,BackupOnline_Nao,BackupOnline_SemServicoDeInternet,BackupOnline_Sim,SeguroNoDispositivo_Nao,SeguroNoDispositivo_SemServicoDeInternet,SeguroNoDispositivo_Sim,SuporteTecnico_Nao,SuporteTecnico_SemServicoDeInternet,SuporteTecnico_Sim,TVaCabo_Nao,TVaCabo_SemServicoDeInternet,TVaCabo_Sim,StreamingDeFilmes_Nao,StreamingDeFilmes_SemServicoDeInternet,StreamingDeFilmes_Sim,TipoDeContrato_DoisAnos,TipoDeContrato_Mensalmente,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta
0,1,0,0,1,0,0,1,29,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
1,0,0,1,0,0,0,34,56,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,1,1,0,2,53,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,45,42,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1
4,0,0,1,1,1,0,2,70,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0


## Verificando o balanceamento dos dados

Dado que o estudo de caso se relaciona a um problema de classificação, no qual deve-se classificar consumidores de incorrerem em cenários de churn ou não, faz-se necessário saber a proporcionalidade dos dados.

Por exemplo, se no presente dataframe haver mais casos de churn positivo em detrimento de negativo, isso pode prejudicar a precisão do modelo, devido ao viés gerado.

Desse modo, uma etapa necessária em todo o processo de classificação é verificar a proporcionalidade dos dados, de modo a analisar se um não está em maior proporção que outro e vice-versa.

In [20]:
# Verificando a proporcionalidade dos dados:

pd.DataFrame(customer_df_2['Churn'].value_counts())

,count
Churn,
0,5140
1,1845


In [21]:
# Calcular a proporção dos dados na coluna 'categoria'
proporcao = customer_df_2['Churn'].value_counts(normalize=True)

# Converter para DataFrame para facilitar o plot
df_proporcao = proporcao.reset_index()
df_proporcao.columns = ['churn', 'proporcao']

# Criar o gráfico de barras usando Plotly
fig = px.bar(df_proporcao, x='churn', y='proporcao',
             title='Proporção dos Churns',
             labels={'proporcao': 'Proporção', 'churn': 'Churn'})

# Redimensionando a figura:
fig.update_layout(
    width=600,
    height=400
)

# Mostrar o gráfico
fig.show()

Analisando tanto a tabela com as proporções geradas quanto o gráfico, nota-se que os dados estão desproporcionais, de modo que há muitos dados que representam um não churn e poucos que sim.

Para que consigamos torná-los proporcionais, pode-se utilizar a ténica de oversampling (sobre-amostragem), que consiste na criação de novas observações da classe em que há menos amostras. A sua vantagem é que ela, ao contrário da técnica Undersampling, que consiste na remoção da quantidade dos dados da classe majoritária, mantém as informações dessa, não havendo, portanto, perda de informação.

Um alerta, porém, acerca dessa abordagem é que ela pode produzir overfitting, cenário no qual o modelo fica sobretreinado, tornando-o muito preciso para a etapa de treino e mais impreciso para generalização na etapa de teste.

Uma das técnicas de oversampling mais utilizada é o SMOTE (Synthetic Minority Over-sampling Technique), que cria exemplos sintéticos da classe minoritária, gerando dados mais realistar que simples duplicação desses.

In [22]:
# Para tornar os dados proporcionais, irei utilizar
# SMOTE, no contexto da abordagem do oversampling.

# Para podermos aplicar o SMOTE, devemos separar  os dados em variáveis características e resposta

# Para utilizá-lo, precisa-se dividir em X e y:
X = customer_df_2.drop('Churn', axis = 1)
y = customer_df_2['Churn']

In [23]:
# Instancia o SMOTE.
smt = SMOTE(random_state=22)

# Realiza a reamostragem do conjunto.
X, y = smt.fit_resample(X, y)

In [24]:
customer_df_2 = pd.concat([X, y], axis = 1)

# Verificando novamente a proporcionalidade
# da variável target:

pd.DataFrame(customer_df_2['Churn'].value_counts())

,count
Churn,
0,5140
1,5140


## K-Nearest Neighbors (KNN)

Modelo de aprendizado supervisionado que pode ser utilizado tanto para classificação quanto para regressão, o qual é baseado na concepção de que itens, dados, termos semelhantes são próximos uns dos outros.

Desse modo, a sua classificação ou previsão reside no fato de que a partir das variáveis explicativas que estarão relacionadas às suas respostas, irá comparar as demais outras. Com base nas suas distâncias, dos valores de cada variável explicativa, ele irá dar uma resposta de classificação ou de previsão para novos dados de entrada que buscam ser classificados ou previstos.

>

Seria como fazer o seguinte :    

`pessoa indie = comportamento*a + estilo*b + gostos*c`

`pessoa gótica = comportamento*a' + estilo*b' + gosto*c'`

`pessoa nornie? = comportamento*a" + estilo*b" + gosto*c"`

>

O modelo iria prever qual estilo da 'pessoa nornie?' com base o quanto de seu comportamento, estilo e gosto está próximo ou distante da pessoa gótica e indie, de modo que quanto mais próxima da indie poderia ser classificada como tal, ou, do contrário, nesse cenário, como gótica.

Portanto, em síntese, o modelo KNN classifica ou preve com base no treinamento, que consiste na "assimilação" (treinamento) de que certas variáveis explicativas se relacionam a certas respostas, de modo a quando for requisitada a prever um dado irá comparar as suas características (distância) com o que fora treinada anteriormente.

>

### Como funciona o KNN ?

- Armazena os dados de treinamento - não há fase de aprendizado propriamente dita ;

- Cálculo da distância - quando um novo dado de entrada é recebido, o modelo calcula a distância entre esse ponto e todos os pontos dos dados de treinamento. A utilização do uso da distância euclidiana para o cálculo é mais comum, mas há outras, como a Manhattan ou Minkowski.

- Identifica os K vizinhos mais próximos - Seleciona os K pontos de dados que estão mais próximos

- Classificação - O novo ponto é classificado, com base na maioria dos votos dos seus K mais próximos, de modo que o rótulo mais comum entre esses é atribuido ao novo dado

- Regressão - O valor predito é a média dos K próximos.

>

### Vantagens ♦️

Possui simplicidade, sendo fácil de se implementar e é adaptável, servindo tanto para classificação quanto para regressão.

### Desvantagens ♣️

- Computacionalmente intensivo :

a fase de predição requer um intenso cálculo da distância entre os pontos, como forma de fazer o modelo prever o novo dado, com base nos seus dados de entrada.

- Sensível a dados ruidosos :   

o modelo justamente pelo seu cálculo de distância é sensível aos dados ruidosos, de modo que a sua precisão pode ser afetada pela existência deles, demandando, para tanto, de normalização, recurso utilizado para colocar os dados sujeitos numa mesma escala, além de reduzir a influência dos outliers no conjunto de dados.



## Distâncias 📏

Sabe-se que o modelo de KNN funciona com base nas distâncias dos dados entre si, incluindo o de nova entrada, que buscam ser classificados ou previstos. Mas dentre as distâncias principais encontradas (euclidiana, manhattan, minkowski, chebyshev).

>

- Euclidiana

>

$[ d(\mathbf{p}, \mathbf{q}) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 +   \cdots + (p_n - q_n)^2}]$

>

Cálcula a menor distância entre dois pontos - portanto uma linha reta.

>

Pode ser usada no KNN para cenários nos quais as variáveis ou dimensões têm unidades semelhantes e importâncias iguais, assim como quando não há no conjunto de dados a presença de muitos outliers.

>

- Manhattan

>

$   d(\mathbf{p}, \mathbf{q}) = \sum_{i=1}^{n} |p_i - q_i| $

>

Calcula a distância percorrida em ângulos retos.

É útil para cenários nos quais os dados apresentam dimensionalidade e importâncias diferentes, bem como unidades de medida dissonantes, além se houver a presença de outliers no conjunto de dados.

>

- Minkowski

A Minkowski é adaptável de modo a por meio do valor p (1 para Manhattan e 2 para Euclidiana) a ela associada, seu parâmetro, transforma a função matemática que a constrói na distância de Manhattan ou na distância Euclidiana.

- Chebyshev

$d(\mathbf{p}, \mathbf{q}) = \max_{i=1}^{n} |p_i - q_i|$

Apresenta movimento restrito a ângulos retos, sendo utilizado quando precisa da máxima distância entre as dimensões. Desse modo, num intervalo de cada dimensão, ele vai se concetrar na dimensão que possuir maior distância. Também é robusto a outliers.


## Implementando o modelo de KNN 👻

In [25]:
# Normalizando os dados:

# Dado que o KNN é um modelo de ML baseado em distância
# para a sua operação, faz-se necessário deixar os dados
# normalizados, sujeito a uma mesma escala, de modo que
# possuam média 0, variando conforme um desvio padrão.

X = customer_df_2.drop('Churn', axis = 1)
y = customer_df_2['Churn']

scaler = StandardScaler()

X_normalizado = scaler.fit_transform(X)

In [26]:
# Segmentando o conjunto de dados em treino / teste :

SEED = 22

X_train, X_test, y_train, y_test = train_test_split(X_normalizado, y,
                                                    test_size = 0.3,
                                                    random_state = SEED)

In [27]:
# Instanciando o modelo e informando a métrica,
# a distância que iremos utilizar. Além dela, poderíamos
# passar a quantidade dos vizinhos a ser considerados para
# a previsão do modelo, o K, uma vez que por padrão ele
# instancia com 5.

# A quantidade de vizinhos é importante, pois a depender
# pode engendrar cenários de overfitting (quando o K é
# muito pequeno) ou em cenários de underfitting (quando o
# K é muito elevado).

knn = KNeighborsClassifier(metric = 'euclidean')

# Treinando o modelo:
knn.fit(X_train, y_train)

# Prevendo dados com o modelo.
knn_pred = knn.predict(X_test)

In [28]:
# Calculando as métricas:

acuracia = accuracy_score(y_test, knn_pred)
f1 = f1_score(y_test, knn_pred)


print(f'O modelo tem uma acurácia de {acuracia.round(3)*100} %')
print(f'O modelo tem um f1-score de {(f1*100).round(2)} %')


O modelo tem uma acurácia de 81.89999999999999 %
O modelo tem um f1-score de 81.93 %


Comparando as duas métricas de avaliação do modelo, compreende-se que, uma vez que o f1-score não é inferior, significativamente, a acurácia, o modelo possui uma boa precisão, revocação e, portanto, taxa de acerto (acurácia).

A f1-score é a média harmônica da precisão e da revocação. A precisão se refere a quanto dos dados classificados como positivos, por exemplo, são realmente positivos, enquanto que a revocação analisa dos casos tidos como positivos são realmente positivos em relação aos dados reais.

Desse modo, a precisão seria a razão de positivos previstos em relação aos positivos do modelo, enquanto que a revocação a razão dos dados tidos como positivos em relação aos dados como um todo do conjunto de dados.

É importante saber que existe uma relação de *trade-off*  entre precisão e revocação, de modo que se definimos um modelo para ser muito preciso, ele poderá ser muito restritivo, desconsiderado alguns dados que poderiam ser positivos, sendo por ele, porém, desconsiderado, na forma em que se busca revocação, pode-se adentrar a um cenário no qual perde-se precisão, deixando o modelo menos restritivo.

## DummyClassifier 🤡

Eu tenho a avaliação do modelo de KNN para o conjunto de dados utilizados, mas o quão bom efetivamente ele é ?

Compará-lo com ele mesmo pode ser pouco informativo, porém compará-los com outros modelos pode ser mais rico. Nesse sentido, uma técnica basilar adotada para comparar o quão bom é um modelo é utilizar um baseline bobo. Isso mesmo, bobo.

Se, por exemplo, a acurácia do modelo bobo for maior ou equivalente ao do nosso modelo, significa que o nosso modelo é no máximo tão bom quanto um modelo bobo.

Na literatura, encontramos esse modelo com o nome DummyClassifier, o qual será utilizado para criarmos o baseline.

In [29]:
# Instanciando o Dummy:
dummy = DummyClassifier(strategy="most_frequent")

# Treine o Dummy Classifier
dummy.fit(X_train, y_train)

# Faça previsões com o Dummy Classifier
pred_dummy = dummy.predict(X_test)

# Calcule a acurácia do Dummy Classifier
acuracia_dummy = accuracy_score(y_test, pred_dummy)
f1_dummy = f1_score(y_test, pred_dummy)

print(f'Acurácia do dummy {acuracia_dummy.round(3)*100} %')
print(f'O dummy tem um f1-score de {(f1_dummy*100).round(2)} %')

Acurácia do dummy 48.4 %
O dummy tem um f1-score de 65.27 %


Observando o modelo Dummy utilizado como baseline ao presente contexto, concebe-se que, sim, o modelo elaborado utilizando o KNN é bom, sendo significativamente superior aos percentuais encontrados pelo Dummy Classifier.

## Bernoulli Naiive Bayes

Modelo que associa a multivariada de Bernoulli com o teorema de Bayes.

A multivariada de Bernoulli é um modelo matemático que mensura a probabilidade de eventos binários ocorrerem, de modo que a probabilidade de um pode influir na de outro. Relaciona-se a dados binários, que podem ser multivariados, portanto.

O teorema de naiive Bayes é um teorema que estuda a probabilidade de um evento ocorrer dado que ocorreu outro. O naiive presente em seu nome deriva dele pressupor que as variáveis explicativas são independentes, ou seja, uma não se relaciona com a outra.

Por exemplo, segundo Bayes, a Mclaren com a qual o Senna ganhou seu bicampeonato, o Senna propriamente dito, a equipe dos mecânicos e projetistas, a probabilidade de todos produzirem um bom trabalho não influi em nenhum outro, bastando, porém, apenas saber a probabilidade independentes dessa para saber se ele seria ou não campeão.

Em outras palavras : " Pouco importa a influência em que essas partes apresentam em relação a si, parte da premissa que não influi, bastando saber a probabilidade de cada qual, para que saiba se o Senna seria ou não campeão ".

Voltando ao modelo, a união de Bernoulli com Bayes, produz um modelo que relaciona a probabilidade da ocorrência de um evento dado que existira outro, de modo que as variáveis sejam independentes ao mesmo tempo que são binárias.

In [30]:
# Uma vez que vou utilizar Bernoulli Naiive Bayes
# para criar um outro modelo de A. de Máquina, para o
# presente estudo de caso, devo conceber a proporção
# das colunas que não são binárias, pois esse modelo é
# criado considerando variáveis explicativas binárias.


# Identificar variáveis binárias
binary_columns = [col for col in customer_df_2.columns if customer_df_2[col].nunique() == 2]
num_binary = len(binary_columns)
total_columns = customer_df_2.shape[1]

# Calcular proporções
proportion_binary = num_binary / total_columns
proportion_non_binary = 1 - proportion_binary

# Mostrar resultados
print(f'Total de colunas: {total_columns}')
print(f'Colunas binárias: {num_binary} ({proportion_binary:.2%})')
print(f'Colunas não binárias: {total_columns - num_binary} ({proportion_non_binary:.2%})')


Total de colunas: 39
Colunas binárias: 37 (94.87%)
Colunas não binárias: 2 (5.13%)


Dado que a proporção de colunas não binárias em relação às binárias é muito inferior 2 para 37, pode-se conceber que podemos 'binarizar' tais variáveis, para utilizar no modelo de Bernoulli Naiive Bayes.

In [31]:
# Para sabermos o valor que servirá como métrica
# de clivagem, por meio da qual o modelo irá binarizar
# a variável, assumindo um valor de 1 ou 0, devo passar
# como o parâmetro um valor.

# Dado que não desejo definir a clivagem em algum quartil,
# é pouco eficiente escolher um valor de modo arbitrário,
# que a média é sensível a outliers, irei selecionar
# a mediana, que além desse aspecto divide o conjunto
# de dados 'no meio'. Desse modo, acima dela os valores
# serão tidos como 1 e abaixo como 0.

np.median(X_train)

-0.43914937080337674

In [32]:
# Instanciando o modelo:
bnb = BernoulliNB(binarize = 0.44)

# Treinando o modelo:
bnb.fit(X_train, y_train)

# Prevendo os dados com o modelo já treinado
predito_Bnb = bnb.predict(X_test)

In [33]:
acuracia_Bnb = accuracy_score(y_test, predito_Bnb)
f1_Bnb = f1_score(y_test, predito_Bnb)


print(f'O modelo tem uma acurácia de {acuracia_Bnb.round(3)*100} %')
print(f'O modelo tem um f1-score de {(f1_Bnb*100).round(2)} %')


O modelo tem uma acurácia de 76.7 %
O modelo tem um f1-score de 78.02 %


## Árvores de decisão 🌳

Modelo de machine learning utilizado tanto para classificação quanto para regressão. O seu funcionamento pode ser visto esquematicamente como a de uma árvore, na medida que apresenta a 'raiz', local da entrada dos dados iniciais, a partir da qual vai segmentando os dados, analisando-os um a um, de modo a classificá-los ou regredi-los, por meio dos nós, que culminam nas folhas, último local de análise no qual a árvore faz a sua última classificação ou regressão.

Não obstante, ainda há conceitualmente aquilo que se denomina por ramos, sendo a combinação dos nós com as folhas. Como a árvore pode aumentar em sua profundidade, não necessariamente um nó já termina numa folha, mas podem existir 'n' nós e folhas até esse processo, formando, portanto, os ramos.

Importante dizer que as árvores podem trabalhar tanto com dados numéricos quanto categóricos.

>

### Estrutura da árvore :     

- Raiz - entrada dos dados ;
- Nó interno - representa uma característica (atributo) do conjunto de dados ;
- Nó folha - representa um resultado (classe ou valor) ;
- Ramo - representa uma regra de decisão, sendo a união do nó interno com o da folha.

>

### Como funciona ?

- Divisão recursiva :     

A árvore realiza divisões como forma de reduzir, a cada passo, o espaço amostral dos dados, de modo a formar instâncias que apresentam valores homogêneos

- Seleção dos atributos :    

A divisão é perpassada pela seleção dos atributos. Em cada passo da seleção, a árvore seleciona o atributo que mais reduz a "impureza", normalmente utilizando métricas como índice de Gini, entropia ou variância - utilizada em cenários de regressão.

Mas o que seriam impurezas ? As impurezas são as misturas que existem entre os dados, as variáveis explicativas que existem na árvore a partir dos dados de entrada. A árvore opera como forma de separar os dados para que existe em uma única folha dados homogêneos, isto é, que possuam um mesmo valor ou classe. Desse modo, compreende-se as impurezas como a mistura de um ou mais variável explicativa (ou atributo) num nó da árvore.

- Critério de parada :     

A divisão continua até que uma condição de parada seja obitida, podendo ser um número mínimo de amostras numa folha ou profundidade máxima de uma árvore.

### Desvantagens

As árvores de decisão são sucetíveis a overfitting, pois podem se tornar muito complexas e realizar um sobretreino com os dados do conjunto. Para evitar isso, utiliza-se convencionalmente de duas principais técnicas : a pós-poda e a pré-poda.

Além disso, pode ser instável, de modo que variações nos dados podem resultar em árvores de decisão distintas, e, quando comparadas a outro modelos, como Random Forest e Gradient Boosting, podem apresentar caráter sub-ótimo.  

## Implementando o modelo 🍀

In [34]:
# Define os parâmetros a serem testados
parametros = {
    'criterion': ['entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Instanciando a árvore de decisão:
dt = DecisionTreeClassifier()

# Usando o GridSearchCV para encontrar os melhores parâmetros com validação cruzada
grid_search = GridSearchCV(dt, parametros, cv=5)

# Treina o modelo com os dados de treino:
grid_search.fit(X_train, y_train)

# Imprime os melhores parâmetros encontrados
print("Melhores Parâmetros:", grid_search.best_params_)

Melhores Parâmetros: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 10}


Explicando a célula acima:

O modelo das árvores de decisão é conhecido por incorrer em cenários de overfitting, cenário no qual o modelo é sobretreinado. Para evitar isso, como já comentado, há a técnica da pré-poda, na qual passa-se como hiperparâmetro valores acerca da profundidade máxima da árvore, mínima quantidade de amostra por nó / folha e assim por diante, e a pós-poda, que, diferente da primeira, instancia primeiramente o modelo com um universo de parâmetros especificados, de modo que, por meio do GridSearch, instancia 'n' árvores com cada um dos parâmetros fornecidos.

Então os melhores valores são retornados, por meio do `grid_search.best_params_`, os quais não produzirão o risco de chegar a um cenário de overfitting.

In [35]:
# Criando a árvore, agora, com os hiperparâmetros encontrados:

dt = DecisionTreeClassifier(criterion= 'entropy', max_depth = 8,
                            min_samples_leaf = 4, min_samples_split = 2,
                            random_state = 22)

dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=4,
                       random_state=22)

In [36]:
dt.feature_importances_

array([0.00667188, 0.00796878, 0.        , 0.00195532, 0.00252219,
       0.12610653, 0.06513313, 0.02074687, 0.        , 0.02252739,
       0.        , 0.05367337, 0.00147043, 0.01042496, 0.        ,
       0.00778917, 0.00193887, 0.        , 0.00424289, 0.00671066,
       0.        , 0.00131329, 0.00341959, 0.        , 0.01316973,
       0.00632663, 0.        , 0.00096094, 0.0134011 , 0.        ,
       0.00747788, 0.07243767, 0.29191906, 0.14773244, 0.02278723,
       0.04254648, 0.01362675, 0.02299877])

In [37]:
dt_pred = dt.predict(X_test)

acuracia_dt = accuracy_score(y_test, dt_pred)
f1_dt = f1_score(y_test, dt_pred)


print(f'O modelo da árvore de decisão tem uma acurácia de {acuracia_dt.round(3)*100} %')
print(f'O modelo da árvore de decisão tem um f1-score de {(f1_dt*100).round(2)} %')

O modelo da árvore de decisão tem uma acurácia de 81.6 %
O modelo da árvore de decisão tem um f1-score de 80.98 %


In [ ]:
from sklearn.metrics import confusion_matrix

## Matriz de confusão 😵‍💫

A utilização da matriz de confusão é útil para conseguir avaliar o modelo criado para além das métricas da acurácia e f1-socore por exemplo, na forma que por mais que ambos modelos possam ter métricas semelhantes, por meio das quais poderia ser considerado que ambos são úteis a resolução da problemática, a matriz permite compreender o quão efetivo são no que se refere a taxa de acerto de verdadeiros positivos / negativos em relação aos falsos positivos / negativos.

### Matriz de confusão para o modelo KNN :

In [48]:
# Obtendo as classes únicas presentes em y_test
classes = sorted(set(y_test))

# Criando a matriz de confusão
cm_knn = confusion_matrix(y_test, knn_pred)

fig = px.imshow(cm_knn,
                 labels=dict(x="Previsto", y="Real", color="Contagem"),
                x=classes,
                y=classes,
                color_continuous_scale='viridis',
                text_auto =  True)

# Ajustando o tamanho e a cor do texto
fig.update_traces(textfont_size=16, textfont_color='blue')

fig.update_layout(title_text="Matriz de Confusão", title_x=0.5)
fig.show()

### Matriz de confusão para o modelo Bernoulli Naive Bayes :

In [50]:
# Criando a matriz de confusão
cm_bnb = confusion_matrix(y_test, predito_Bnb)

fig = px.imshow(cm_bnb,
                 labels=dict(x="Previsto", y="Real", color="Contagem"),
                x=classes,
                y=classes,
                color_continuous_scale='viridis',
                text_auto =  True)

# Ajustando o tamanho e a cor do texto
fig.update_traces(textfont_size=16, textfont_color='blue')

fig.update_layout(title_text="Matriz de Confusão", title_x=0.5)


### Matriz de confusão para árvores de decisão :

In [51]:
# Criando a matriz de confusão
cm_dt = confusion_matrix(y_test, dt_pred)

fig = px.imshow(cm_dt,
                 labels=dict(x="Previsto", y="Real", color="Contagem"),
                x=classes,
                y=classes,
                color_continuous_scale='viridis',
                text_auto =  True)

# Ajustando o tamanho e a cor do texto
fig.update_traces(textfont_size=16, textfont_color='blue')

fig.update_layout(title_text="Matriz de Confusão", title_x=0.5)


Considerando as matrizes acima, compreende-se que os modelos mais efetivos são tanto o KNN quanto as árvores de decisão, na forma que possuem taxa de acerto para verdadeiros e falsos positivos / negativos semelhante. O modelo criado pelo Bernoulli Naive Bayes, por mais que apresente uma taxa de acurácia e f1-score próxima, por outro lado, apresenta mais falsos positivos e negativos, revelando ser mais impreciso que os outros citados.